In [1]:
import pandas as pd

In [2]:
from entsoe import EntsoePandasClient
from entsoe.exceptions import InvalidBusinessParameterError, NoMatchingDataError
from requests import HTTPError

In [3]:
def get_balancing(zone_list):
    
    unavailable_zones = []
    
    balancing = pd.DataFrame()

    for zone in zone_list:
        
        print("Extracting balancing of zone " + zone)
        
        for business_type in business_types:
            
            bal = pd.DataFrame()

            try: 
                act = client.query_activated_balancing_energy(zone, start=start, end=end, business_type=business_type)            
                act = act.resample("1h").sum().unstack()            
                act.name = business_type
                bal = pd.concat([bal, act],axis=1)

            except: # (HTTPError, NoMatchingDataError, InvalidBusinessParameterError):
                unavailable_zones.append(zone)
        
        if len(bal)>0:
            bal = bal.sum(axis=1)
            bal.name = zone 
            balancing = pd.concat([balancing, bal], axis=1)
            
    if len(unavailable_zones) >0:
        print("Unable to retrieve " + ", ".join(unavailable_zones) + " from ENTSO-E transparency" )
    
    
    balancing.reset_index(0, drop=True, inplace=True)

    balancing.loc[
        ("Down", balancing.index.levels[1]), balancing.columns
    ] = -balancing.loc[("Down", balancing.index.levels[1]), balancing.columns]

    return balancing
            
        


In [4]:
api_key = "99516228-fb84-4a73-9b38-d0703b72cce0"
client = EntsoePandasClient(api_key=api_key)

In [5]:
start = pd.Timestamp("2018-01-01", tz="Europe/Brussels")
end = pd.Timestamp("2022-01-01", tz="Europe/Brussels")

In [26]:
ordc_hdf = "data/ordc_parameters.h5"

In [6]:
business_types = [ 'A96',  'A97']

In [7]:
reserve_requirements = pd.read_excel("data/pemmdb.xlsx", "Reserve Requirements",index_col=[0,1])[["FCR (MW)", "FRR (MW)"]].sum(axis=1)

In [8]:
reserve_requirements = reserve_requirements[reserve_requirements >0]

In [9]:
reserve_requirements.index = reserve_requirements.index.remove_unused_levels()

In [10]:
bz_map = pd.Series(
    reserve_requirements.index.levels[0].str[:2],
    index = reserve_requirements.index.levels[0]
)

In [11]:
bz_map.drop(["UKNI", "NOM1", "NOS0", "NON1"], inplace=True) # norway to be built separately because ERAA contains only 3 NO zones 
                                                            # while ENTSO-E transparency features 5 zones.

In [12]:
bz_map.loc[
    ["DKW1", "DKE1", "SE01", "SE02", "SE03", "SE04", "DE00", 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI']
] = ["DK_1","DK_2", 'SE_1', 'SE_2', 'SE_3', 'SE_4', 'DE_LU','IT_CALA', 
     'IT_CNOR', 'IT_CSUD',  'IT_NORD',  'IT_SUD' ,  'IT_SARD', 'IT_SICI']

In [13]:
norway = pd.Series(
    ["NOS0", "NOS0", "NOM1", "NON1", "NOS0"],
    ['NO_1', 'NO_2', 'NO_3', 'NO_4', 'NO_5'] 
)

In [14]:
balancing = get_balancing(bz_map.values)

Extracting balancing of zone AL
Extracting balancing of zone AT
Extracting balancing of zone BA
Extracting balancing of zone BE
Extracting balancing of zone BG
Extracting balancing of zone CH
Extracting balancing of zone CY
Extracting balancing of zone CZ
Extracting balancing of zone DE_LU
Extracting balancing of zone DK_2
Extracting balancing of zone DK_1
Extracting balancing of zone EE
Extracting balancing of zone ES
Extracting balancing of zone FI
Extracting balancing of zone FR


Connection Error, retrying in 0 seconds


Extracting balancing of zone GR
Extracting balancing of zone HR
Extracting balancing of zone HU
Extracting balancing of zone IE
Extracting balancing of zone IT_CALA
Extracting balancing of zone IT_CNOR
Extracting balancing of zone IT_CSUD
Extracting balancing of zone IT_NORD
Extracting balancing of zone IT_SUD
Extracting balancing of zone IT_SARD
Extracting balancing of zone IT_SICI
Extracting balancing of zone LT
Extracting balancing of zone LV


Connection Error, retrying in 0 seconds


Extracting balancing of zone MK
Extracting balancing of zone MT
Extracting balancing of zone NL
Extracting balancing of zone PL
Extracting balancing of zone PT
Extracting balancing of zone RO
Extracting balancing of zone RS
Extracting balancing of zone SE_1
Extracting balancing of zone SE_2
Extracting balancing of zone SE_3
Extracting balancing of zone SE_4
Extracting balancing of zone SI
Extracting balancing of zone SK
Extracting balancing of zone UK
Unable to retrieve BG, BG, CH, CY, CY, DK_2, EE, GR, GR, IE, IE, IT_CALA, IT_CALA, IT_CNOR, IT_CNOR, IT_CSUD, IT_CSUD, IT_NORD, IT_NORD, IT_SUD, IT_SUD, IT_SARD, IT_SARD, IT_SICI, IT_SICI, LT, LV, MK, MK, MT, MT, PL, UK, UK from ENTSO-E transparency


In [15]:
balancing_mean = balancing.stack().groupby(
    pd.Series(bz_map.index, bz_map.values)
    .reindex([i[2] for i in balancing.stack().index]).values
).mean()

In [16]:
balancing_std = balancing.stack().groupby(
    pd.Series(bz_map.index, bz_map.values)
    .reindex([i[2] for i in balancing.stack().index]).values
).std()

In [17]:
balancing_norway = get_balancing(norway.index)

Extracting balancing of zone NO_1
Extracting balancing of zone NO_2
Extracting balancing of zone NO_3
Extracting balancing of zone NO_4
Extracting balancing of zone NO_5


In [18]:
balancing_mean = pd.concat(
    [
        balancing_mean, 
        balancing_norway.stack().groupby(
            norway.reindex([i[2] for i in balancing_norway.stack().index]).values
        ).mean()
    ]
)

In [19]:
balancing_std = pd.concat(
    [
        balancing_std, 
        balancing_norway.stack().groupby(
            norway.reindex([i[2] for i in balancing_norway.stack().index]).values
        ).std()
    ]
)

In [20]:
balancing_mean = balancing_mean.reindex(reserve_requirements.index.levels[0], fill_value=0)

In [21]:
balancing_std = balancing_std.reindex(reserve_requirements.index.levels[0])

In [22]:
missing_zones = balancing_std.loc[balancing_std.isna()].index

In [23]:
scale_missing = reserve_requirements.loc[missing_zones].groupby(level=0).mean().div(
    reserve_requirements.groupby(level=0).mean().mean()
)

In [24]:
balancing_std.loc[missing_zones] = scale_missing.multiply(balancing_std.mean())

In [40]:
ordc_parameters = pd.concat([balancing_mean, balancing_std], axis=1)
ordc_parameters.columns = ["mean", "std"]
ordc_parameters.to_hdf(ordc_hdf, "ordc_parameters")